# <font color="red"> Natural Language Processing with Disaster Tweets

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import warnings
warnings.filterwarnings("ignore")

import nltk
import textblob
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

from textblob import TextBlob, Word
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split

In [2]:
#!pip install stop-words
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#pip install transformers

In [4]:
df1 = pd.read_csv("train.csv")

In [5]:
df1

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [6]:
df1.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [7]:
df2=pd.read_csv("test.csv")

In [8]:
df2

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [9]:
df=pd.concat([df1,df2])

In [10]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,NaN
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,NaN
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [11]:
df[:7613]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1.0
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1.0
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1.0
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1.0


In [12]:
del df["keyword"]
del df["location"]

In [13]:
df.isnull().sum()

id           0
text         0
target    3263
dtype: int64

In [14]:
df["text"] = df["text"].str.lower()
df["text"] = df["text"].str.replace('[^a-zA-ZğüşıöçĞÜŞİÖÇ\s]', ' ',regex=True)
df["text"] = df["text"].str.replace('[^\w\s]',' ',regex=True) #noktalama işaretleri ve gereksiz karakterleri kaldırır
df["text"] = df["text"].str.replace('\n',' ') 
df["text"] = df["text"].str.replace('\d+',' ',regex=True) #sayıları boşluk ile değiştirir
df["text"] = df["text"].str.replace('https?://\S+|www\.\S+', ' ',regex=True)
df["text"] = df["text"].str.replace('\r',' ')
df["text"] = df["text"].str.replace('_',' ')

import re

df["text"] = df["text"].apply(lambda x: re.sub("[^a-zA-Z0-9\s]","",x)) #alfa-numerik olamayan karakterleri kaldırma

In [15]:
df["text"]

0       our deeds are the reason of this  earthquake m...
1                  forest fire near la ronge sask  canada
2       all residents asked to  shelter in place  are ...
3              people receive  wildfires evacuation or...
4       just got sent this photo from ruby  alaska as ...
                              ...                        
3258    earthquake safety los angeles     safety faste...
3259    storm in ri worse than last hurricane  my city...
3260    green line derailment in chicago http   t co u...
3261    meg issues hazardous weather outlook  hwo  htt...
3262     cityofcalgary has activated its municipal eme...
Name: text, Length: 10876, dtype: object

In [16]:
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
pr=PorterStemmer()

In [17]:
def lemmafn(text):
    words=TextBlob(text).words
    return [pr.stem(word) for word in words]

In [18]:
vect=CountVectorizer(stop_words='english', analyzer=lemmafn)

In [19]:
train=df[:7613].copy()

In [20]:
test=df[7613:].copy()

In [21]:
train

,id,text,target
0,1,our deeds are the reason of this earthquake m...,1.0
1,4,forest fire near la ronge sask canada,1.0
2,5,all residents asked to shelter in place are ...,1.0
3,6,people receive wildfires evacuation or...,1.0
4,7,just got sent this photo from ruby alaska as ...,1.0
...,...,...,...
7608,10869,two giant cranes holding a bridge collapse int...,1.0
7609,10870,aria ahrary thetawniest the out of control w...,1.0
7610,10871,m utc km s of volcano hawaii htt...,1.0
7611,10872,police investigating after an e bike collided ...,1.0


In [22]:
test

,id,text,target
0,0,just happened a terrible car crash,NaN
1,2,heard about earthquake is different cities s...,NaN
2,3,there is a forest fire at spot pond geese are...,NaN
3,9,apocalypse lighting spokane wildfires,NaN
4,11,typhoon soudelor kills in china and taiwan,NaN
...,...,...,...
3258,10861,earthquake safety los angeles safety faste...,NaN
3259,10865,storm in ri worse than last hurricane my city...,NaN
3260,10868,green line derailment in chicago http t co u...,NaN
3261,10874,meg issues hazardous weather outlook hwo htt...,NaN


In [23]:
x=train["text"]
y=train["target"]

In [24]:
y= y.astype("int64")

In [25]:
x=vect.fit_transform(x) #tokenize etme işini de halletmiş oluyoruz

In [26]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [27]:
from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import AdaBoostClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier

In [29]:
from numpy.lib.shape_base import column_stack
classifiers=[
    LogisticRegression(),
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    MultinomialNB(),
    BernoulliNB()]

col_title=["Classifier","Methods","Accuracy","Precision","F1","Recall"]
amazon=pd.DataFrame(columns=col_title)

for i in classifiers:
  pred=i.fit(x_train,y_train).predict(x_test)
  name=i.__class__.__name__

  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred,average="weighted")
  f1=f1_score(y_test,pred,average="weighted")
  recall=recall_score(y_test,pred,average="weighted")

  result = pd.DataFrame([[name,"CountVectorizer",accuracy,precision,f1,recall]],columns=col_title)
  amazon=amazon.append(result)

amazon

,Classifier,Methods,Accuracy,Precision,F1,Recall
0,LogisticRegression,CountVectorizer,0.804990,0.804953,0.803154,0.804990
0,KNeighborsClassifier,CountVectorizer,0.690085,0.727894,0.655373,0.690085
0,DecisionTreeClassifier,CountVectorizer,0.732764,0.731390,0.731684,0.732764
0,AdaBoostClassifier,CountVectorizer,0.763624,0.762872,0.760950,0.763624
0,GradientBoostingClassifier,CountVectorizer,0.768877,0.774263,0.762320,0.768877
0,MultinomialNB,CountVectorizer,0.804334,0.803636,0.803358,0.804334
0,BernoulliNB,CountVectorizer,0.809586,0.811527,0.806716,0.809586


### BernoulliNB

In [30]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [31]:
bNB = BernoulliNB()
model=bNB.fit(x_train,y_train)
pred = model.predict(x_test)
accuracy_score(pred, y_test)

0.8095863427445831

In [32]:
model

BernoulliNB()

In [33]:
pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

### Submission

In [52]:
a=test["text"]
vect=CountVectorizer(stop_words='english', analyzer=lemmafn)
a = vect.fit_transform(a)

In [53]:
a

<3263x10899 sparse matrix of type '<class 'numpy.int64'>'
	with 53340 stored elements in Compressed Sparse Row format>

In [54]:
x

<7613x18995 sparse matrix of type '<class 'numpy.int64'>'
	with 123791 stored elements in Compressed Sparse Row format>

In [56]:
bNB = BernoulliNB()
model=bNB.fit(x,y)
pred = model.predict(a)
accuracy_score(pred, test["text"])

ValueError: X has 10899 features, but BernoulliNB is expecting 18995 features as input.